In [47]:
import pandas as pd
import pymysql
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup
import re


class DBUpdater:
    def __init__(self):
        '''생성자 : MariaDB 연결 및 종목코드 딕셔너리 생성'''
        self.conn = pymysql.connect(host='localhost', user='root', password='dkstjdcks1206', db='INVESTAR', charset='utf8')
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
            code VARCHAR(20),
            company VARCHAR(40),
            last_update DATE,
            PRIMARY KEY (code))
            """
            curs.execute(sql)
            
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
            code VARCHAR(20),
            date DATE,
            open BIGINT(20),
            high BIGINT(20),
            low BIGINT(20),
            close BIGINT(20),
            diff BIGINT(20),
            volume BIGINT(20),
            PRIMARY KEY (code, date))
            """
            curs.execute(sql)
            
        self.conn.commit()
        
        self.codes = dict()
        self.update_comp_info()
        
    def __del__(self):
        '''소멸자: MariaDB 연결 해제'''
        self.conn.close()
        
    def read_krx_code(self):
        '''krx로부터 상장법인목록 파일을 읽어와서 데이터프레임으로 반환'''
        krx = pd.read_html('C:/Users/201910810/workspace/Study/stock/data/상장법인목록.xls', header=0)[0]
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드':'code', '회사명':'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx
    
    def update_comp_info(self):
        '''종목코드를 company_info 테이블에 업데이트한 후 딕셔너리에 저장'''
        sql = "SELECT * FROM company_info" 
        df = pd.read_sql(sql, self.conn) # company_info(종목코드 저장) 테이블 읽어옴
        for idx in range(len(df)): #df 열 수만큼 읽은 다음
            self.codes[df['code'].values[idx]]=df['company'].values[idx] # code(종목코드)와 company dict 형태로 매칭
        with self.conn.cursor() as curs: 
            sql = "SELECT max(last_update) FROM company_info" # company_info에서 last_update max(제일 최신 업데이트 날짜)
            curs.execute(sql)
            rs = curs.fetchone() # 가져옴
            today = datetime.today().strftime('%Y-%m-%d')
            
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today: # 날짜가 존재하지 않거나 오늘보다 오래됨경우
                krx = self.read_krx_code() # df 불러와 서
                for idx in range(len(krx)):
                    code = krx.code.values[idx] 
                    company = krx.company.values[idx]
                    sql = f"REPLACE INTO company_info (code, company, last_update)  VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql) # code, company, today 날짜 업데이트
                    self.codes[code] = company # dict에 code : company 매칭. 어따씀?
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] {idx:04d} REPLACE INTO company_info VALUES({code}, {company}, {today})")
                
            self.conn.commit()
            print('')
            
    def read_naver(self, code, company, pages_to_fetch):
        '''네이버 금융에서 주식 시세를 읽어서 데이터프레임으로 반환'''
        try:
            # 페이지 수 알아오기
            url = f'https://finance.naver.com/item/sise_day.nhn?code={code}' # 종목 하나의 전체 시세
            driver = webdriver.Chrome(executable_path="C:/chromedriver.exe")
            driver.get(url)
            req = driver.page_source
            if req is None:
                return None
            soup = BeautifulSoup(req, "html.parser")
            pgrr = soup.find('td', class_ = 'pgRR') # page Right Right
            if pgrr is None:
                return None
            lastpage = str(pgrr.a['href']).split('=')[-1]
            
            
            # 주식 시세를 읽어서 데이터프레임으로 반환
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch) # pages_to_fetch??
            for page in range(1, pages+1):
                page_url = '{}&page={}'.format(url, page)
                driver.get(page_url)
                df = df.append(pd.read_html(driver.page_source, header=0)[0], ignore_index = True)
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.format(tmnow, company, code, page, pages), end="\r")
            df = df.rename(columns = {'날짜':'Date', '시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})
            df['date'] = df['date'].replace('.','-')          
            df = df.dropna().reset_index().drop('index', axis = 1)
            df[['close','diff','open','high','low','volume']] = df[['close','diff','open','high','low','volume']].astype(int)
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df
            
        
    def replace_into_db(self, df, num, code, company):
        '''네이버 금융에서 읽어온 주식 시세를 DB에 replace '''
        with self.conn.cursor() as curs:
            for r in df.itertuples(): # tuples
                sql = f"REPLACE INTO daily_price VALUES('{code}, "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.closw},"\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.comit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                 'price [OK]'.format(datetime.now().strftime('%Y-%m-%d %H:%M'), num+1, company, code, len(df)))
            
        
    def update_daily_price(self, pages_to_fetch):
        '''krx 상장법인의 주식 시세를 네이버로부터 읽어서 db에 업데이트'''
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])
        
        
    def execute_daily(self):
        '''실행 즉시 및 매일 오후 다섯시에 daily_price 테이블 업데이트'''
        
if __name__ == '__main__':
    dbu = DBUpdater()
    dbu.update_comp_info()

TypeError: update_daily_price() missing 1 required positional argument: 'pages_to_fetch'

In [34]:
df

NameError: name 'df' is not defined

In [13]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')
            

'2021-12-27'

In [33]:
len(dbu.codes)

2487

In [41]:
dbu.codes

{'396770': '엔에이치스팩22호',
 '397880': '교보11호스팩',
 '400840': '하이제7호스팩',
 '298870': '케이티비네트워크',
 '403360': '예스피치',
 '254160': '제이엠멀티',
 '404990': '신한서부티엔디리츠',
 '199800': '툴젠',
 '396690': '미래에셋글로벌리츠',
 '397500': '대신밸런스제11호스팩',
 '402420': '켈스',
 '402340': 'SK스퀘어',
 '377480': '마인즈랩',
 '361570': '알비더블유',
 '400760': 'NH올원리츠',
 '222160': '바이옵트로',
 '290090': '트윔',
 '372800': '아이티아이즈',
 '311320': '지오엘리먼트',
 '376300': '디어유',
 '357880': '비트나인',
 '389030': '지니너스',
 '377300': '카카오페이',
 '376180': '피코그램',
 '348370': '엔켐',
 '377450': '리파인',
 '382800': '지앤비에스엔지니어링',
 '114840': '아이패밀리에스씨',
 '248020': '젬',
 '261780': '차백신연구소',
 '382480': '지아이텍',
 '391710': '엔에이치스팩21호',
 '393210': '토마토시스템',
 '381970': '케이카',
 '376290': '씨유테크',
 '159010': '아스플로',
 '382840': '원준',
 '388220': '하나금융19호스팩',
 '391060': '엔에이치스팩20호',
 '257720': '실리콘투',
 '260970': '에스앤디',
 '377220': '프롬바이오',
 '099430': '바이오플러스',
 '393360': '신한제8호스팩',
 '387310': '대신밸런스제10호스팩',
 '329180': '현대중공업',
 '395400': 'SK리츠',
 '388800': '유진스팩7호',
 '273640': '와이엠텍

In [15]:
import pymysql
pymysql.connect(host='localhost', user='root', password='dkstjdcks1206', db='INVESTAR', charset='utf8')

In [17]:
sql = f"REPLACE INTO company_info (code, company, last_update)  VALUES ('{'a'}', '{'b'}', '{'c'}')"

In [48]:
a = [1, 2]

In [49]:
a.pop(1)

2

In [50]:
a

[1]